In [2]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler
from cs285.envs.pendulum.pendulum_env import PendulumEnv
from cs285.envs.dt_sampler import ConstantSampler
from cs285.infrastructure.replay_buffer import ReplayBufferTrajectories
from cs285.infrastructure.utils import sample_n_trajectories, RandomPolicy
from cs285.agents.ode_agent import ODEAgent
from cs285.agents.utils import save_leaves, load_leaves
from cs285.infrastructure import utils
from typing import Callable, Optional, Tuple, Sequence
import numpy as np
import gym
from cs285.infrastructure import pytorch_util as ptu
from tqdm import trange
import jax
import jax.numpy as jnp
import equinox as eqx
import diffrax
from diffrax import diffeqsolve, Dopri5
import optax
import pickle
from tqdm import trange

In [3]:
key = jax.random.PRNGKey(0)
from jax.lib import xla_bridge
print(xla_bridge.get_backend().platform)

gpu


In [13]:
dt_sampler = ConstantSampler(dt=0.05)
env = PendulumEnv(
    dt_sampler=dt_sampler
)
mpc_dt_sampler = ConstantSampler(dt=0.05)
agent_key, new_agent_key = jax.random.split(key)
neural_ode_name = "vanilla"
neural_ode_kwargs = {
    "ode_dt0": 0.005,
    "mlp_dynamics_setup": {
        "hidden_size":128,
        "num_layers":4,
        "activation":"tanh",
        "output_activation":"identity"
    }
}
optimizer_name = "adamw"
optimizer_kwargs = {"learning_rate": 1e-3}
mb_agent = ODEAgent(
    env=env,
    key=agent_key,
    neural_ode_name=neural_ode_name,
    neural_ode_kwargs=neural_ode_kwargs,
    optimizer_name=optimizer_name,
    optimizer_kwargs=optimizer_kwargs,
    ensemble_size=10,
    train_discount=1,
    mpc_horizon_steps=20,
    mpc_dt_sampler=mpc_dt_sampler,
    mpc_strategy="cem",
    mpc_discount=0.9,
    mpc_num_action_sequences=1000,
    cem_num_iters=4,
    cem_num_elites=5,
    cem_alpha=1,
)

In [ ]:
replay_buffer = ReplayBufferTrajectories(seed=42)
trajs, _ = sample_n_trajectories(env, RandomPolicy(env=env), ntraj=1000, max_length=200, key=key)
replay_buffer.add_rollouts(trajs)

In [4]:
"""
with open("random_replay_buffer", "wb") as f:
    pickle.dump(replay_buffer, f)
"""

'\nwith open("random_replay_buffer", "wb") as f:\n    pickle.dump(replay_buffer, f)\n'

In [14]:
batch_size = 16
train_ep_len = 20
train_stride = 20
train_steps = 100

In [15]:
all_losses = []
for step in trange(train_steps, dynamic_ncols=True):
    step_losses = []
    for i in range(mb_agent.ensemble_size):
        traj = replay_buffer.sample_rollouts(batch_size=batch_size)
        obs = utils.split_arr(np.array(traj["observations"]), length=train_ep_len, stride=train_stride)
        acs = utils.split_arr(np.array(traj["actions"]), length=train_ep_len, stride=train_stride)
        dts = utils.split_arr(np.array(traj["dts"])[..., np.newaxis], length=train_ep_len, stride=train_stride).squeeze(-1)
        batch_size, num_splitted, train_ep_len, ob_dim = obs.shape
        ac_dim = acs.shape[-1]
        obs = jnp.array(obs).reshape(batch_size * num_splitted, train_ep_len, ob_dim)
        acs = jnp.array(acs).reshape(batch_size * num_splitted, train_ep_len, ac_dim)
        times = jnp.cumsum(dts, axis=-1).reshape(batch_size * num_splitted, train_ep_len)
        loss = mb_agent.batched_update(
            i=i,
            obs=obs, 
            acs=acs, 
            times=times
        )
        step_losses.append(loss)
    save_leaves(mb_agent, f"checkpoint/checkpoint{step}")
    all_losses.append(np.mean(step_losses))
    print(f"epoch {step}, loss {np.mean(step_losses)}")

  1%|          | 1/100 [00:20<33:09, 20.10s/it]

epoch 0, loss 18.39639196395874


  2%|▏         | 2/100 [00:39<32:20, 19.80s/it]

epoch 1, loss 16.3913480758667


  3%|▎         | 3/100 [00:59<32:04, 19.84s/it]

...

epoch 97, loss 3.2696501493453978


 99%|█████████▉| 99/100 [36:19<00:22, 22.48s/it]

epoch 98, loss 3.5318663239479067


100%|██████████| 100/100 [36:41<00:00, 22.02s/it]

epoch 99, loss 3.576762342453003


In [ ]:
# problem: low gpu utilization rate, need investigation

In [ ]:
save_leaves(mb_agent, "vanilla_trained_on_500_random_steps")

In [8]:
mb_agent = load_leaves(mb_agent, "checkpoint_161")

(ScaleByAdamState(count=Array(164, dtype=int32), mu=NeuralODE_Vanilla(
   mlp=MLP(
     layers=(
       Linear(
         weight=f32[128,4],
         bias=f32[128],
         in_features=4,
         out_features=128,
         use_bias=True
       ),
       Linear(
         weight=f32[128,128],
         bias=f32[128],
         in_features=128,
         out_features=128,
         use_bias=True
       ),
       Linear(
         weight=f32[128,128],
         bias=f32[128],
         in_features=128,
         out_features=128,
         use_bias=True
       ),
       Linear(
         weight=f32[128,128],
         bias=f32[128],
         in_features=128,
         out_features=128,
         use_bias=True
       ),
       Linear(
         weight=f32[3,128],
         bias=f32[3],
         in_features=128,
         out_features=3,
         use_bias=True
       )
     ),
     activation=None,
     final_activation=None,
     use_bias=True,
     use_final_bias=True,
     in_size=4,
     out_size=3,
  